In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statistics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

df=pd.read_csv('tiki.csv')
df.head()

,ID,Name Product,Price(VND),Total Review,Discount(%)
0,0,Máy Hút Bụi Mini Cầm Tay DEERMA DX128C,699000,166,58.0
1,1,Máy Lọc Không Khí Sharp FP-J30E-B - Hàng chí...,1799000,70,54.0
2,2,"Combo 3 lõi lọc nước Karofi 1,2,3 chính hãng",120000,586,57.0
3,3,Bàn Ủi Khô Nagakawa NAG1502 - Màu Ghi - Hàng C...,200000,1,13.0
4,4,Máy Hút Bụi Cầm Tay Không Dây Damas XC628 - Ch...,379000,318,58.0


In [2]:
df['Discount(%)'].isnull().sum()  # đếm số giá trị thiếu trong cột

1

In [4]:
df['Price(VND)'].isnull().sum() 

0

In [ ]:
def Outliers(choice,old_data):
    data=old_data.copy() # Tránh mất dữ liệu 
    if choice == 0: # Không xử lý ngoại lệ
        data[['Price(VND)','Discount(%)']] 
    elif choice == 1: # Xử lý ngoại lệ 
        data.loc[data['Age'] > 73,'Age']=73
        data.loc[data['Fare']> 100,'Fare']=100
    new_data=data[['Age','Fare']] # dữ liệu trả về
    return new_data

In [ ]:
def Draw(new_data, name) :
    old_data = df.copy()
    fig = plt.figure()
    ax = fig.add_subplot(111)
    old_data['Age'].plot(kind='kde', color='blue')
    new_data['Age'].plot(kind='kde', color='red')
    lines, labels = ax.get_legend_handles_labels()
    ax.legend(lines, labels, loc='best')
    plt.title(name + " Age")

In [ ]:
def MissingValues(choice , data):
    new_data = data.copy()  # Tránh mất dữ liệu 
    new_data2 = df.copy()
    if choice==0: #Mean
        mean=new_data['Age'].mean()
        new_data['Age'].fillna(mean,inplace=True)
        Draw(new_data,"Mean")       
        
    elif choice==1: #median
        median=new_data['Age'].median()
        new_data['Age'].fillna(median,inplace=True)        
        Draw(new_data,"Median")
       
    elif choice==2: #mode
        mode_imputation=statistics.mode(df['Age'])
        new_data['Age'].fillna(mode_imputation,inplace=True)
        Draw(new_data,"Mode")

    elif choice==3: #Random
        random_samples = new_data['Age'].dropna().sample(n = new_data['Age'].isnull().sum(),random_state=10)
        random_samples.index = new_data2[new_data2['Age'].isnull()].index
        new_data.loc[new_data['Age'].isnull(), 'Age'] = random_samples
        Draw(new_data,"Random")

    elif choice==4: #Tail
        extreme = new_data.Age.mean() + 3*new_data.Age.std()  # trung bình + phân bố chuẩn 
        new_data['Age'].fillna(extreme,inplace=True)
        Draw(new_data,"Tail")
        
    elif choice==5: # Random bat ki
        
        random=np.random.uniform(data.Age.min(), data.Age.max())
        random=round(random, 1)    
        print(random)
        new_data['Age'].fillna(random,inplace=True)
        Draw(new_data,"Arbitrary")
        
    elif choice==6: # New Features
        new_data['Age_NAN']=np.where(df['Age'].isnull(),1,0)
        new_data['Age'].fillna(df['Age'].median(),inplace=True)
        Draw(new_data,"New Features")
    plt.show()
    return new_data

In [ ]:
def Predict_Score(new_data):
    list_accuracy_score = []
    for i in range (10):
        X_train,X_test,y_train,y_test=train_test_split(new_data, df['Survived'], 
                                                       test_size=0.3, random_state=i)
        classifier=LogisticRegression()
        classifier.fit(X_train,y_train)
        y_pred=classifier.predict(X_test)
        list_accuracy_score.append(accuracy_score(y_test,y_pred))
    return np.mean(list_accuracy_score)

In [ ]:
Table={'Type':['Mean','Median','Mode','Random Values','Tail Values','Arbitrary Values','New Features'],
                      'Not Outliers':np.zeros((7,),dtype=float),
                      'Outliers':np.zeros((7,),dtype=float)}

Table=pd.DataFrame(Table)

for i in range (7):
    for j in range(2):
        temp = Outliers(j,df.copy())
        temp = MissingValues(i,temp)
        Table.iloc[i,j+1] = Predict_Score(temp)